# 라이브러리 설치

In [15]:
# -*- coding: utf-8 -*-

!pip install catboost
!pip install -U scikit-learn
!pip install numpy
!pip install pandas

# Import

In [2]:
import os
import random
import pandas as pd
import numpy as np
import sklearn
import catboost

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from datetime import datetime

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

# Data Load

In [4]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

# Data Pre-processing

In [5]:
#TIMESTAMP 피쳐를 datetime으로 나누기
tr_month =[]
tr_day = []
tr_hour =[]
tr_minute = []
for i in range(train_df.shape[0]):
  tr_month.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').month)
  tr_day.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').day)
  tr_hour.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').hour)
  tr_minute.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').minute)

train_df['MONTH'] = tr_month
train_df['DAY'] = tr_day
train_df['HOUR'] = tr_hour
train_df['MINUTE'] = tr_minute

te_month =[]
te_day = []
te_hour =[]
te_minute = []
for i in range(test_df.shape[0]):
  te_month.append(datetime.strptime(test_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').month)
  te_day.append(datetime.strptime(test_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').day)
  te_hour.append(datetime.strptime(test_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').hour)
  te_minute.append(datetime.strptime(test_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').minute)

test_df['MONTH'] = te_month
test_df['DAY'] = te_day
test_df['HOUR'] = te_hour
test_df['MINUTE'] = te_minute

In [6]:
# TIMESTAMP 피쳐 제거
train_df=train_df.drop(columns=['TIMESTAMP'])
test_df=test_df.drop(columns=['TIMESTAMP'])

#전부 NAN값인 피쳐 제거
a=[]
A = pd.DataFrame(train_df.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_df.shape[0]].index)
train_df.drop(columns=a,inplace=True)
test_df.drop(columns=a, inplace=True)

#분산이 1인 피쳐 제거
a=[]
for c in train_df.columns:
  if  train_df[c].nunique()==1:
    a.append(c)
train_df.drop(columns=a, inplace=True)
test_df.drop(columns=a, inplace=True)

In [7]:
# 3,4번 라인 피쳐 분석
groups_lines=train_df.groupby('LINE')
line=0
for name, group in groups_lines:
    line+=1
    globals()['group_line{}_df'.format(line)] = groups_lines.get_group(name)
non_null_counts_3 = group_line3_df.notnull().sum()
non_null_cols_3 = non_null_counts_3[non_null_counts_3 > 0].index.tolist()

grouped3 = group_line3_df.groupby(group_line3_df.isnull().apply(tuple, axis=1))
cnt=0
for name, group in grouped3:
    cnt+=1
    globals()['line3_{}_df'.format(cnt)] = grouped3.get_group(name)
for i in range(1,cnt+1):
  globals()['non_null_cols3_{}'.format(i)] = globals()['line3_{}_df'.format(i)].columns[globals()['line3_{}_df'.format(i)].notnull().all()].tolist()
  del globals()['non_null_cols3_{}'.format(i)][globals()['non_null_cols3_{}'.format(i)].index('PRODUCT_ID')]

grouped4 = group_line4_df.groupby(group_line4_df.isnull().apply(tuple, axis=1))
cnt=0
for name, group in grouped4:
    cnt+=1
    globals()['line4_{}_df'.format(cnt)] = grouped4.get_group(name)
for i in range(1,cnt+1):
  globals()['non_null_cols4_{}'.format(i)] = globals()['line4_{}_df'.format(i)].columns[globals()['line4_{}_df'.format(i)].notnull().all()].tolist()
  del globals()['non_null_cols4_{}'.format(i)][globals()['non_null_cols4_{}'.format(i)].index('PRODUCT_ID')]

# 3번라인과 4번라인 피쳐들의 합집합구하기
col_34=[]
for i in non_null_cols3_3:
  if i not in col_34:
    col_34.append(i)
for j in non_null_cols4_1:
  if j not in col_34:
    col_34.append(j)

# 3,4번 라인 튜닝용 데이터 셋 저장 
col_34_train = col_34
train_x = train_df[col_34_train]
col_34_test = col_34
del col_34_test[col_34_test.index('Y_Quality')]
del col_34_test[col_34_test.index('Y_Class')]
test_x = test_df[col_34_test]

train_x34 = train_x[train_x['LINE'].isin(['T050304','T050307'])]
test_x34 = test_x[test_x['LINE'].isin(['T050304','T050307'])]

In [8]:
# 전체 학습용 데이터 결측값 전처리
train_x = train_df.drop(columns=['PRODUCT_ID'])
test_x = test_df.drop(columns=['PRODUCT_ID'])
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [9]:
qual_col = ['LINE', 'PRODUCT_CODE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 

train_y = train_x['Y_Quality']
train_yy = train_x['Y_Class']
train_x.drop(columns=['Y_Quality','Y_Class'],inplace=True)

In [10]:
for i,c in enumerate(train_x34.columns[~train_x34.columns.isin(['Y_Quality','Y_Class','LINE','PRODUCT_CODE'])]):
  train_x34[c].fillna(0,inplace=True)
  test_x34[c].fillna(0,inplace=True)

yy_train34 = train_x34['Y_Class']
train_x34 = train_x34.drop(columns=['Y_Class', 'Y_Quality','LINE','PRODUCT_CODE'])
test_x34 = test_x34.drop(columns=['LINE','PRODUCT_CODE'])

# Model Fit & Inference

In [11]:
# regressor
reg = catboost.CatBoostRegressor(learning_rate=0.05, iterations=500, verbose=0)
reg.fit(train_x, train_y, early_stopping_rounds=100, cat_features=['PRODUCT_CODE', 'LINE'])

pre_preds = reg.predict(test_x)

a = train_df[['Y_Class','Y_Quality']].groupby('Y_Class').agg(['mean', 'min', 'max', 'count'])

preds=[]
for p in pre_preds:
  if p<=a[('Y_Quality','max')][0]:
    preds.append(0)
  elif p<=a[('Y_Quality','min')][2]:
    preds.append(1)
  else:
    preds.append(2)

RR = test_x

In [12]:
# 3,4 line classifier
clf = catboost.CatBoostClassifier(random_seed=37,iterations=500, verbose=0)
clf.fit(train_x34, yy_train34)
y_preds = clf.predict(test_x34)

a = RR[(RR['LINE']==2)|(RR['LINE']==3)].index
temp = preds
for i in range(len(a)):
  temp[a[i]]=y_preds[i][0]

# Submit

In [13]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = temp
submit.to_csv('./sample_submission.csv', index=False)